In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import time

interval = 86400 #seconds                                                                                                                
now = 1505573241#int(time.time() - 60*60) * 1000  #epoch seconds                                                                        
past = now - interval

chicago_index = "network_weather-2017*"

es_chicago = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) # Chicago instance                                                

query_chicago = {
    "_source": [   "timestamp", "srcSite", "destSite", "delay_mean"  ],
    "query": {
        "bool": {
        "must": [
            { "range": {  "timestamp": { "gte": past,  "lte": now,  "format": "epoch_second"  }  } },
            { "term" : {"_type" : "latency"}},
            { "term" : {"srcVO" : "ATLAS"}},
            { "term" : {"destVO" : "ATLAS"}}
        ]  }  }
}

scroll_chicago = scan(client=es_chicago, query=query_chicago, index=chicago_index, scroll='5m', timeout='5m', size=10000, filter_path=['_scroll_id', '_shards', 'hits.hits._source'])

counter = 0
start_time = time.time()
for entry in scroll_chicago:
    print (entry)
    break
    if not counter%100000:  print("Processing event number ", counter)
    counter += 1

print("%s entries processes in %i seconds" %(counter, time.time() - start_time))

{'_source': {'delay_mean': 41.05011666666667, 'destSite': 'UKI-SCOTGRID-ECDF', 'srcSite': 'RU-Protvino-IHEP', 'timestamp': 1505530527000}}
0 entries processes in 0 seconds
